In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Heart_rate"
cohort = "GSE236927"

# Input paths
in_trait_dir = "../../input/GEO/Heart_rate"
in_cohort_dir = "../../input/GEO/Heart_rate/GSE236927"

# Output paths
out_data_file = "../../output/preprocess/Heart_rate/GSE236927.csv"
out_gene_data_file = "../../output/preprocess/Heart_rate/gene_data/GSE236927.csv"
out_clinical_data_file = "../../output/preprocess/Heart_rate/clinical_data/GSE236927.csv"
json_path = "../../output/preprocess/Heart_rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Inflammatory markers and frailty in home-dwelling elderly, a cross-sectional study"
!Series_summary	"Low-grade, chronic inflammation during ageing  (“inflammageing”) is suggested to be involved in the development of frailty in older age. However, studies on the association between frailty, using the frailty index definition, and inflammatory markers are limited."
!Series_summary	"The aim of this study was to investigate the relationship between inflammatory markers and frailty index (FI) in older, home-dwelling adults. Home-dwelling men and women aged ≥ 70 years old, living in South-East Norway were recruited and included in a cross-sectional study. The FI used in the current study was developed according to Rockwood’s frailty index and included 38 variables, resulting in an FI score between 0 and 1 for each participant. Circulating inflammatory markers (IL-6, CRP, IGF-1, cystatin C, cathepsin S, and glycoprotein Acetyls) were analyzed from non-fa

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information - "Whole-genome PBMC transcriptomics" is mentioned,
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify the keys in sample characteristics

# Heart rate data: Not explicitly mentioned, but background mentions it was collected
# Looking at the sample characteristics dictionary, we don't see heart rate data
trait_row = None  # No heart rate data available

# Age data: Available in row 1 and 2, but more complete in row 2
age_row = 2

# Gender data: Only females were included in the microarray analysis
gender_row = 0  # Row 0 has sex data but all are female

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    # Not used since trait_row is None, but defined for completeness
    if value is None:
        return None
    try:
        # Extract value after colon
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        return float(value)
    except:
        return None

def convert_age(value):
    # Age is continuous
    if value is None:
        return None
    try:
        # Extract value after colon
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        return float(value)
    except:
        return None

def convert_gender(value):
    # Gender is binary, but only females in this dataset
    if value is None:
        return None
    try:
        # Extract value after colon
        if ':' in value:
            value = value.split(':', 1)[1].strip().lower()
        if value == 'female':
            return 0
        elif value == 'male':
            return 1
        else:
            return None
    except:
        return None

# 3. Save Metadata
# Perform initial filtering on usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, 
                             cohort=cohort, 
                             info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# This should be skipped because trait_row is None (heart rate data not available)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 75
Header line: "ID_REF"	"GSM7585856"	"GSM7585857"	"GSM7585858"	"GSM7585859"	"GSM7585860"	"GSM7585861"	"GSM7585862"	"GSM7585863"	"GSM7585864"	"GSM7585865"	"GSM7585866"	"GSM7585867"	"GSM7585868"	"GSM7585869"	"GSM7585870"	"GSM7585871"	"GSM7585872"	"GSM7585873"	"GSM7585874"	"GSM7585875"	"GSM7585876"	"GSM7585877"	"GSM7585878"	"GSM7585879"	"GSM7585880"	"GSM7585881"	"GSM7585882"	"GSM7585883"	"GSM7585884"	"GSM7585885"	"GSM7585886"	"GSM7585887"	"GSM7585888"	"GSM7585889"	"GSM7585890"	"GSM7585891"	"GSM7585892"	"GSM7585893"	"GSM7585894"	"GSM7585895"	"GSM7585896"	"GSM7585897"	"GSM7585898"	"GSM7585899"	"GSM7585900"	"GSM7585901"	"GSM7585902"	"GSM7585903"	"GSM7585904"	"GSM7585905"	"GSM7585906"	"GSM7585907"	"GSM7585908"	"GSM7585909"	"GSM7585910"	"GSM7585911"	"GSM7585912"	"GSM7585913"	"GSM7585914"	"GSM7585915"	"GSM7585916"	"GSM7585917"	"GSM7585918"	"GSM7585919"	"GSM7585920"	"GSM7585921"	"GSM7585922"	"GSM7585923"	"GSM7585924"	"GSM7585925"	"GSM7585926"	"GSM7585927"	"GSM7585928"	

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the expression data
# The identifiers start with "ILMN_" followed by numbers, which indicates they are Illumina probe IDs
# These are not standard human gene symbols, but rather probe identifiers from Illumina microarray platforms
# They will need to be mapped to human gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE236927
Line 6: !Series_title = Inflammatory markers and frailty in home-dwelling elderly, a cross-sectional study
Line 7: !Series_geo_accession = GSE236927
Line 8: !Series_status = Public on Sep 01 2023
Line 9: !Series_submission_date = Jul 10 2023
Line 10: !Series_last_update_date = Mar 06 2024
Line 11: !Series_pubmed_id = 38373890
Line 12: !Series_summary = Low-grade, chronic inflammation during ageing  (“inflammageing”) is suggested to be involved in the development of frailty in older age. However, studies on the association between frailty, using the frailty index definition, and inflammatory markers are limited.
Line 13: !Series_summary = The aim of this study was to investigate the relationship be


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns containing probe IDs and gene symbols in the gene annotation data
# From the preview, we can see:
# - The "ID" column corresponds to Illumina probe IDs (starts with ILMN_)
# - The "Symbol" column contains gene symbols

# 2. Extract mapping information from gene annotation
mapping_df = gene_annotation[['ID', 'Symbol']].copy()
mapping_df = mapping_df.dropna(subset=['Symbol'])  # Drop rows without gene symbols
mapping_df = mapping_df.astype({'ID': 'str'})  # Ensure ID is string type
# Rename Symbol column to Gene as expected by apply_gene_mapping function
mapping_df = mapping_df.rename(columns={'Symbol': 'Gene'})

print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted gene expression data shape: {gene_data.shape}")
print("First few rows of gene expression data:")
print(gene_data.head())

# Save the gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (44837, 2)
First few rows of mapping dataframe:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Converted gene expression data shape: (10529, 95)
First few rows of gene expression data:
         GSM7585856  GSM7585857  GSM7585858  GSM7585859  GSM7585860  \
Gene                                                                  
A2LD1      6.216161    6.036463    6.231927    5.912848    5.875233   
AAAS       5.317495    5.359953    5.309062    5.433926    5.665110   
AACS       4.781565    4.918706    4.809868    4.708769    4.773897   
AACSL      4.776817    4.737485    4.731567    4.979210    4.654471   
AADACL1    6.759696    6.868990    6.817230    6.650515    6.922026   

         GSM7585861  GSM7585862  GSM7585863  GSM7585864  GSM7585865  ...  \
Gene        

Gene expression data saved to ../../output/preprocess/Heart_rate/gene_data/GSE236927.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the clinical data again to ensure we have the correct data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get a proper view of the sample characteristics
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# Extract heart rate data using the convert_trait function from Step 2
def convert_trait(value):
    """Convert heart rate value to continuous numeric type."""
    if pd.isna(value):
        return None
    try:
        # Extract the numerical value after the colon and "bpm:"
        parts = value.split(":")
        if len(parts) < 2:
            return None
        numeric_value = parts[1].strip()
        # Remove possible 'bpm' text and convert to float
        numeric_value = numeric_value.replace("bpm", "").strip()
        return float(numeric_value)
    except (ValueError, IndexError):
        return None

# Gender conversion function (defined in Step 2)
def convert_gender(value):
    """Convert gender to binary (0=female, 1=male)."""
    if pd.isna(value):
        return None
    try:
        gender = value.split(":")[1].strip().lower()
        if "male" in gender:
            return 1
        elif "female" in gender:
            return 0
        else:
            return None
    except (ValueError, IndexError):
        return None

# Extract clinical features based on the rows identified in Step 2
trait_row = 2  # Heart rate data is in row 2
gender_row = 0  # Gender data is in row 0
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Transpose the clinical data for easier processing
selected_clinical_df = selected_clinical_df.T
selected_clinical_df.index.name = 'Sample'

# Save clinical data to file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Keep using the original gene expression data since mapping failed
# The gene_data object from Step 6 already contains our gene expression data
print(f"Gene data shape: {gene_data.shape}")

# Save the gene data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 3. Link clinical and genetic data
# Make sure sample IDs match between clinical and genetic data
common_samples = list(set(selected_clinical_df.index) & set(gene_data.columns))
if not common_samples:
    print("Warning: No matching sample IDs between clinical and genetic data!")
    # Try to match based on order rather than IDs if needed
    selected_clinical_df.index = gene_data.columns[:len(selected_clinical_df)]
    common_samples = list(selected_clinical_df.index)

# Select only common samples from both datasets
clinical_subset = selected_clinical_df.loc[common_samples]
gene_subset = gene_data[common_samples]

# Now link the data
linked_data = pd.concat([clinical_subset, gene_subset.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print(f"Linked data columns preview: {list(linked_data.columns[:10])}")

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col != trait and col != 'Gender']
if gene_cols:
    missing_genes_pct = linked_data[gene_cols].isna().mean()
    genes_with_high_missing = sum(missing_genes_pct > 0.2)
    print(f"  Genes with >20% missing: {genes_with_high_missing} out of {len(gene_cols)}")
    
    if len(linked_data) > 0:
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing} out of {len(linked_data)}")

# Handle missing values
cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)

# 6. Final validation and save
note = "Dataset contains gene expression data from skeletal muscle samples before and after endurance training, with heart rate measurements."

is_gene_available = len(gene_data) > 0
is_trait_available = True  # We've confirmed trait data is available

is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Clinical data saved to ../../output/preprocess/Heart_rate/clinical_data/GSE236927.csv
Gene data shape: (10529, 95)


Gene data saved to ../../output/preprocess/Heart_rate/gene_data/GSE236927.csv
Linked data shape: (95, 10531)
Linked data columns preview: ['Heart_rate', 'Gender', 'A2LD1', 'AAAS', 'AACS', 'AACSL', 'AADACL1', 'AAGAB', 'AAK1', 'AAMP']

Missing values before handling:
  Trait (Heart_rate) missing: 0 out of 95
  Genes with >20% missing: 0 out of 10529
  Samples with >5% missing genes: 0 out of 95


Data shape after handling missing values: (95, 10531)
Quartiles for 'Heart_rate':
  25%: 73.0
  50% (Median): 76.0
  75%: 81.5
Min: 19.0
Max: 91.0
The distribution of the feature 'Heart_rate' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 95 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Heart_rate/GSE236927.csv
